# Dependent Parameter and Additional Outputs

In [1]:
from besos import eppy_funcs as ef
import besos.sampling as sampling
from besos.problem import EPProblem
from besos.evaluator import EvaluatorEP
from besos.parameters import wwr, RangeParameter, DependentParameter, Parameter, FieldSelector
from besos.optimizer import NSGAII

## Dependent Parameter

The dependent parameter couples two parameters together: Target parameter with Leading value (set by the sampling or optimization algorithm), and dependent parameter with following (reacts) value. 

In the following example, we want to set the sum of the ElectricEquipment and Light to 25. Assume ElectricEquipment is the target parameter, and Light is the DependentParameter.

In [2]:
parameters = []
parameters.append(wwr())  #index 0
parameters.append(
    Parameter(
        name='Electric Equipment',
        selector=FieldSelector(
            class_name="ElectricEquipment",
            object_name="*",
            field_name="Watts per Zone Floor Area",
        ),
        value_descriptor=RangeParameter(min_val=10, max_val=15)
    )
)  #index 1

Now we create a DependentParameter for Light. A DependentParameter requires three parameters: mode, factor, and index. 

There are three modes for DependentParameter:

- 'sum': response_value = factor - leading_value 
- 'multiple': response_value = leading_value * factor
- 'power': response_value = leading_value ** factor

The response_value is the used value for the dependent parameter.

For this example, we set the mode to 'sum', and factor to 25.

The index represents the index of the target parameter in the parameter list. For this example, the leading parameter, ElectricEquipment, is at index 1. 

The DependentParameter should be placed after the target parameter in the parameter list. 

In [3]:
parameters.append(
    Parameter(
        name='Lights',
        selector=FieldSelector(
            class_name="Lights",
            object_name="*",
            field_name="Watts per Zone Floor Area",
        ),
        value_descriptor=DependentParameter(mode='sum', factor=25, index=1)
    )
)

In [4]:
objectives = ['Electricity:Facility']
problem = EPProblem(parameters, objectives)
building = ef.get_building()
evaluator = EvaluatorEP(problem, building)

In [5]:
inputs = sampling.dist_sampler(sampling.lhs, problem, 5)
inputs

,Window to Wall Ratio,Electric Equipment,Lights
0,0.486001,14.170300,10.829700
1,0.631623,12.174691,12.825309
2,0.142207,10.586064,14.413936
3,0.311364,13.454279,11.545721
4,0.921740,11.637009,13.362991


## Additional Outputs for optimization algorithm

When running the optimization algorithm, all objectives will be optimized. The result of the objectives in the add_outputs list will include in the data frame, but they will not be optimized.

For the following example, the output is still Electricity:Facility. We also want to know the value of Cooling:Electricity for different solutions, but we don't need it to be optimized. So instead of adding it in objectives, we add it in add_outputs.

First of all we want to check if the 'Cooling:Electricity' is one of the outputs for this building.

In [6]:
from besos.eplus_funcs import print_available_outputs

print_available_outputs(building, name='Cooling:Electricity')

['Cooling:Electricity', 'Hourly']


Now we re-define the problem with add_outputs.

In [7]:
problem = EPProblem(parameters, objectives, add_outputs=['Cooling:Electricity'])
evaluator = EvaluatorEP(problem, building)

results = NSGAII(evaluator, evaluations=1, population_size=1)
results

,Window to Wall Ratio,Electric Equipment,Lights,Electricity:Facility,violation,pareto-optimal,Cooling:Electricity
0,0.197708,13.554301,11.445699,2.020056e+09,0,True,3.853115e+08
